# Image testing for pre-processing

## Importing Modules

In [4]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import csv
import json

## Loading image

In [4]:
img_1 = cv.imread('Collected Leaf data/Our Collection/Natural/IMG_20230922_171310509.jpg')
cv.imshow('Raw Image', img_1)
cv.waitKey(0)
cv.destroyAllWindows()

## Image Smoothening

In [5]:
def GaussianSmoothing(mat):
    _pi = 3.14
    _sd = np.std(mat)
    _e = 2.718
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            try:
                mat[y][x] = ((1 / (2 * _pi * (_sd ** 2)))) * (_e ** (((x ** 2) + (y ** 2)) * (1 / (2 * (x ** 2)))))
            except ZeroDivisionError:
                pass
    return mat

In [6]:
# img_1_myGblur = GaussianSmoothing(cv.cvtColor(img_1, cv.COLOR_BGR2GRAY))

# img_1_myGblur_resized = cv.resize(img_1_myGblur, (1000, 1000))
# cv.imshow('Image Gaussian Blur & Resized', img_1_myGblur_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [7]:
# img_1_resized_Gblur = cv.GaussianBlur(img_1_resized, (7,7), 0)
# cv.imshow('Image Resized & Gaussian Blur', img_1_resized_Gblur)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur = cv.GaussianBlur(img_1, (7,7), 0)
cv.imshow('Image Gaussian Blur', img_1_Gblur)
cv.waitKey(0)
cv.destroyAllWindows()

## Image Resizing

In [8]:
# img_1_resized = cv.resize(img_1, (int(img_1.shape[1] / 6), int(img_1.shape[0] / 6)))
# cv.imshow('Image Resized', img_1_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur_resized = cv.resize(img_1_Gblur, (1000, 1000))
cv.imshow('Image Gaussian Blur & Resized', img_1_Gblur_resized)
cv.waitKey(0)
cv.destroyAllWindows()

In [9]:
img_1_Gblur_resized_gray = cv.cvtColor(img_1_Gblur_resized, cv.COLOR_BGR2GRAY)
cv.imshow('Image Gaussian Blur & Resized & Gray', img_1_Gblur_resized_gray)
cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
# Function to map each intensity level to output intensity level. 
def pixelVal(pix, r1, s1, r2, s2): 
	if (0 <= pix and pix <= r1): 
		return (s1 / r1)*pix 
	elif (r1 < pix and pix <= r2): 
		return ((s2 - s1)/(r2 - r1)) * (pix - r1) + s1 
	else: 
		return ((255 - s2)/(255 - r2)) * (pix - r2) + s2 

# Define parameters. 
r1 = 110 # controls darkness
s1 = 0.15 # lower bound
r2 = 7 # controls lightness
s2 = 250 # upper bound

# Vectorize the function to apply it to each value in the Numpy array. 
pixelVal_vec = np.vectorize(pixelVal) 

# Apply contrast stretching. 
img_1_contrast_stretched = pixelVal_vec(img_1_Gblur_resized_gray, r1, s1, r2, s2) 

cv.imshow('Contrast Stretching', img_1_contrast_stretched)
cv.waitKey(0)
cv.destroyAllWindows()


In [11]:
img_thresh = cv.threshold(img_1_Gblur_resized_gray, 127, 255, cv.THRESH_BINARY)
cv.imshow('Thresholding', img_thresh[1])
cv.waitKey(0)
cv.destroyAllWindows()

# print(img_thresh)

In [12]:
img_hist = cv.equalizeHist(img_thresh[1])
cv.imshow('Histogram equlization', img_hist)
cv.waitKey(0)
cv.destroyAllWindows()

## Combining these processes in a single function

In [13]:
def ImagePreprocess(name: str):
    cv.imshow('Final Image', pixelVal_vec(cv.resize(cv.GaussianBlur(cv.imread(name)
                                           , (7,7), 0), 
                           (1000, 1000)), 
                r1, s1, r2, s2))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [14]:
ImagePreprocess('Collected Leaf data/Our Collection/Natural/IMG_20230922_171255218.jpg')

In [15]:
ImagePreprocess('Collected Leaf data/Our Collection/Natural/IMG20230922165200.jpg')

In [16]:
ImagePreprocess('Collected Leaf data/Our Collection/Natural/IMG20230922170108.jpg')

In [17]:
ImagePreprocess('Collected Leaf data/Our Collection/Natural/IMG20230922170842.jpg')

In [19]:
ImagePreprocess('Collected Leaf data/Botany Collection/Natural/IMG20231004155721.jpg')

## Getting name of the images in a list.

In [43]:
from os import listdir
from os.path import isfile, join
myPath = "Collected Leaf data/Botany Collection/Natural/"

def getImageNameList(path: str) -> list:
    return [f for f in listdir(path) if isfile(join(path, f))]

In [44]:
image_name_lst = getImageNameList(myPath)
image_name_lst

['IMG20231004154634.jpg',
 'IMG20231004154651.jpg',
 'IMG20231004154748.jpg',
 'IMG20231004154809.jpg',
 'IMG20231004154917.jpg',
 'IMG20231004155118.jpg',
 'IMG20231004155132.jpg',
 'IMG20231004155149.jpg',
 'IMG20231004155213.jpg',
 'IMG20231004155323.jpg',
 'IMG20231004155327.jpg',
 'IMG20231004155554.jpg',
 'IMG20231004155602.jpg',
 'IMG20231004155631.jpg',
 'IMG20231004155642.jpg',
 'IMG20231004155721.jpg',
 'IMG20231004155801.jpg',
 'IMG20231004155912.jpg',
 'IMG20231004160002.jpg',
 'IMG20231004160043.jpg',
 'IMG20231004160236.jpg',
 'IMG20231004160249.jpg',
 'IMG20231004160505.jpg',
 'IMG20231004160553.jpg',
 'IMG20231004160623.jpg',
 'IMG20231004160753.jpg',
 'IMG20231004160822.jpg',
 'IMG20231004161319.jpg',
 'IMG20231004161328.jpg',
 'IMG20231004161554.jpg',
 'IMG20231004161653.jpg',
 'IMG20231004161729.jpg',
 'IMG20231004162025.jpg',
 'IMG20231004162125.jpg',
 'IMG20231004162157.jpg',
 'IMG20231004162413.jpg',
 'IMG20231004162427.jpg',
 'IMG20231004162702.jpg',
 'IMG2023100

In [3]:
fields = ['Image', 'Common name', 'Scientific name', 'Class number']

In [13]:
classes = [
    ['Peepla', 'Ficus religiosa'],
    ['Borsalli', 'Mimusops elengi'],
    ['Kamani', 'Murraya paniculata'],
    ['Jasud', 'Hibiscus rosa sinensis'],
    ['Bamboo', 'Bambusa vulguris'],
    ['Clusters fig', 'Ficus racemosa'],
    ['Wedelia', 'Wedelia chinensis'],
    ['Rosy trumpet tree', 'Tabebuia rosea'],
    ['Ixora', 'Ixora coccinea'],
    ['Nagchampo', 'Plumeria pudica'],
    ['Jatropha', 'Jatropha integerrima'],
    ['Aralia', 'Aralia variegated'],
    ['Kadam', 'Neolamarckia cadamba'],
    ['Asopalav', 'Polyalthia longifolium'],
    ['Weeping fig', 'Ficus benjamina'],
    ['Tulsi', 'Ocimum sanctum'],
    ['Jambu', 'Syzygium cumini'],
    ['Karanj', 'Pongamia pinnata'],
    ['Quick stick', 'Gliricidia sepium'],
    ['Pipal', 'Ficus amplissima']
]

classes_json_data = dict(zip(range(len(classes)), classes))

with open('Classes.json', 'w') as Jfile:
    json.dump(classes_json_data, Jfile, indent=4)

In [29]:
def mappingLeafData(path: str, imageNameLst: list) -> list:
    mappedRows = list()
    
    with open('Classes.json', 'r') as Jfile:
        _classes = json.load(Jfile)
    
    for imageName in imageNameLst:
        row = list()
        
        # Show image
        image = cv.imread(path + imageName)
        plt.imshow(image)
        plt.show()

        # Display classes
        for index in range(len(_classes)):
            index = str(index)
            print(index, ")", _classes[index][0], "/", _classes[index][1])

        choice = input("\nEnter choice: ")
        
        row.append(imageName)
        row.append(_classes[choice][0])
        row.append(_classes[choice][1])
        row.append(int(choice))

        mappedRows.append(row)
        # if len(mappedRows) == 1: break
    return mappedRows

## Mapping images of 'Botany Collection/Natural/'

In [39]:
# rows = mappingLeafData(myPath, image_name_lst)

In [40]:
with open('Leaf_Data_Mapped.csv', 'w', newline='') as csvfile:
	# creating a csv writer object 
	csvwriter = csv.writer(csvfile)
		
	# writing the fields 
	csvwriter.writerow(fields)
		
	# writing the data rows 
	csvwriter.writerows(rows)

"""
Mapping completed and stored in Lead_Data_Mapped.csv file
"""


'\nMapping completed and stored in Lead_Data_Mapped.csv file\n'